In [1]:
import torch
from torchvision import models
import numpy as np
from utils import *

In [3]:
alexnet = models.alexnet()

In [3]:
TVLoss(alexnet,[0])

tensor(5.4470, grad_fn=<DivBackward0>)

In [3]:
TVLossMat(alexnet, [1])

tensor(0.0259, grad_fn=<DivBackward0>)

In [5]:
TVLossMat(alexnet, [0])

tensor(0.0544, grad_fn=<DivBackward0>)

In [4]:
TVLossMat(alexnet)

tensor(0.0224, grad_fn=<DivBackward0>)

In [2]:
resnet = models.resnet18()

In [3]:
resnet.conv1

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [44]:
list(resnet.layer1[0].children())[0]

Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [105]:
def TVLossMatResNet(model, layer_mask=None):
    
    blocks = [model.conv1, model.layer1,model.layer2,model.layer3,model.layer4]
    if layer_mask is None:
        layer_mask = [i for i in range(len(blocks))]
    
    # get tv values for convolutions in resnet block
    def TVBlock(block):
        block = list(block.children())
        pixels = 0
        tv = 0
        weights = list(block[0].parameters())[0]
        size = weights.size()
        pixels += size[0] * size[1] * (size[2] - 1) * (size[3] - 1)
        tv += TVMat(weights)
        
        weights = list(block[3].parameters())[0]
        size = weights.size()
        pixels += size[0] * size[1] * (size[2] - 1) * (size[3] - 1)
        tv += TVMat(weights)
        
        return tv, pixels
        
    tv = 0
    pixels = 0
    for layer in layer_mask:
        if layer == 0:
            weights = list(blocks[layer].parameters())[0]
            size = weights.size()
            pixels += size[0] * size[1] * (size[2] - 1) * (size[3] - 1)
            tv += TVMat(weights)
        else:
            basic_blocks = list(blocks[layer].children())
            for bl in basic_blocks:
                curr_tv, curr_pixels = TVBlock(bl)
                pixels += curr_pixels
                tv += curr_tv
    return tv/pixels

def TVMat(weights):
    x = torch.zeros_like(weights)
    y = torch.zeros_like(weights)
    x[:, :, :-1, :-1] = (weights[:, :, 1:, :-1] - weights[:, :, :-1, :-1]) ** 2
    y[:, :, :-1, :-1] = (weights[:, :, :-1, 1:] - weights[:, :, :-1, :-1]) ** 2
    return torch.sum(torch.sqrt(x+y))

In [106]:
TVLossMatResNet(resnet,[0])

tensor(0.0435, grad_fn=<DivBackward0>)

In [107]:
TVLossMatResNet(resnet,[1])

tensor(0.1029, grad_fn=<DivBackward0>)

In [108]:
TVLossMatResNet(resnet,[2])

tensor(0.0726, grad_fn=<DivBackward0>)

In [109]:
TVLossMatResNet(resnet,[3])

tensor(0.0514, grad_fn=<DivBackward0>)

In [98]:
131072/32768

4.0

In [99]:
resnet.layer3

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample): Sequential(
      (0): Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): BasicBlock(
    (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(256, eps=1e-05, mome